In [1]:
from automatic_variable_mapping import corpus, vocab_similarity
import pandas as pd
from functools import partial
import time
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/harrison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/harrison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from automatic_variable_mapping.vocab_similarity import default_pairable, partition
num_cpus = 14

In [3]:
reload(corpus)

<module 'automatic_variable_mapping.corpus' from 'automatic_variable_mapping/corpus.pyc'>

In [37]:
def matching_groups(data, group_col, corpora, pair_id, ref_id):
    corpus_doc_ids = [doc_id for doc_id, _ in corpora]
    ref_idx = corpus_doc_ids.index(ref_id)
    pair_idx = corpus_doc_ids.index(pair_id)
    return data[pair_idx][group_col] == data[ref_idx][group_col]


def pairable_by_group(data, group_col, corpus_doc_ids, score, pair_id, _, ref_id):
    return vocab_similarity.default_pairable(score, pair_id, None, ref_id) and not matching_groups(data, group_col, corpus_doc_ids, pair_id, ref_id)


def calc_score_results(data_file, doc_cols, doc_col_key, ref_id_col, filter_file, mult_corpora=False, corpora_col=None):
    print data_file
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    if mult_corpora:
        corpora_data = partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data

    #idc = list(doc_cols)
    #idc.append(ref_id_col)
    #print corpora_data[0][idc]
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    tfidf_matrix = corpus.calc_tfidf(corpora)

    scores = vocab_similarity.VariableSimilarityCalculator(filter_data[ref_id_col],
                                                           pairable=default_pairable)

    scores.init_cache()
    if mult_corpora:
        result = scores.score_variables(corpora, tfidf_matrix)
        tfidf_label = "multtfcdf"
    else:
        result = scores.score_variables(corpora[0], tfidf_matrix)
        tfidf_label = "tfidf"
    result = result[result["score_desc"] != 0]
    
    result.to_csv("~/Dropbox/tiff_laura_shared/var_txt_sim_" + tfidf_label + "_scores_obs_heart_studies_" + doc_col_key + ".csv")
    return None

In [38]:
obs_data_file = "~/Dropbox/tiff_laura_shared/var_doc_obs_heart_studies_dbGaP_NLP.csv"
obs_man_file = "~/Dropbox/tiff_laura_shared/manual_concept_var_mappings_dbGaP_obs_heart_studies_NLP.csv"
ref_id_col = 'dbGaP_studyID_datasetID_varID'
doc_cols_inputs = {'desc': ['variable_description'],
                   'units': ['units'],
                   'coding': ['variable_coding_counts_distribution'],
                   'desc_units': ['variable_description', 'units'],
                   'desc_coding': ['variable_description', 'units', 'variable_coding_counts_distribution'],
                   'desc_units_coding': ['variable_description', 'units', 'variable_coding_counts_distribution']
                  }


In [39]:
dt = pd.read_csv(obs_man_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
dt
dt.columns

Index([u'study', u'cohort_dbGaP', u'dbGaP_studyID_datasetID',
       u'dbGaP_dataset_label', u'concept', u'data_desc', u'varID', u'var_desc',
       u'units', u'var_coding_counts_distribution', u'var_doc_id',
       u'concept_id', u'dbGaP_studyID_datasetID_varID', u'var_coding_labels'],
      dtype='object')

In [43]:
def calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, filter_file, mult_corpora=False, corpora_col=None):
    scores_dfs = list()
    for key in doc_cols_inputs:
        score_name = "score_" + key
        data = pd.read_csv(data_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
        scores_df = calc_score_results(data_file, doc_cols_inputs[key], key, ref_id_col, filter_file, mult_corpora, corpora_col)
        scores_df = scores_df.rename({'score': score_name}, axis=1)
        scores_dfs.append(scores_df)

    #scores_merged = reduce(lambda left, right: pd.merge(left, right, on=[ref_id_col],
    #                                               how='outer'), scores_dfs)
    #return(scores_merged)
    return(scores_dfs)

In [44]:
obs_scores_tfidf = calc_scores_doc_cols(obs_data_file, doc_cols_inputs, ref_id_col, obs_man_file)

~/Dropbox/tiff_laura_shared/var_doc_obs_heart_studies_dbGaP_NLP.csv


automatic_variable_mapping/corpus.py:83: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  corpus = list (tqdm.tqdm(pool.imap(partial(helper, doc_col), corpus_data[cols].as_matrix()), total=corpus_data.shape[0]))
100%|██████████| 105611/105611 [00:34<00:00, 3048.36it/s]


Finding valid pair ids
Pair ids: 105611
Finding valid ref ids
Ref ids: 1709
Multiplying matrices
LHS: (105611, 60261)
RHS: (1709, 60261)
Sim Matrix: (1709, 105611)
Finding matches for 0 1 phs000007.v26.pht000009.v2.MF5
Finding matches for 1 15 phs000007.v26.pht000009.v2.MF20
Finding matches for 2 557 phs000007.v26.pht000009.v2.shareid
Finding matches for 3 708 phs000007.v26.pht000010.v3.shareid
Finding matches for 4 709 phs000007.v26.pht000011.v3.FB8
Finding matches for 5 716 phs000007.v26.pht000011.v3.FB18
Finding matches for 6 717 phs000007.v26.pht000011.v3.FB19
Finding matches for 7 718 phs000007.v26.pht000011.v3.FB20
Finding matches for 8 719 phs000007.v26.pht000011.v3.FB21
Finding matches for 9 720 phs000007.v26.pht000011.v3.FB22
Finding matches for 10 721 phs000007.v26.pht000011.v3.FB23
Finding matches for 11 726 phs000007.v26.pht000011.v3.FB28
Finding matches for 12 727 phs000007.v26.pht000011.v3.FB29
Finding matches for 13 728 phs000007.v26.pht000011.v3.FB30
Finding matches for

Finding matches for 133 3171 phs000007.v26.pht000020.v3.shareid
Finding matches for 134 3179 phs000007.v26.pht000021.v3.FL021
Finding matches for 135 3180 phs000007.v26.pht000021.v3.FL022
Finding matches for 136 3192 phs000007.v26.pht000021.v3.FL291
Finding matches for 137 3193 phs000007.v26.pht000021.v3.FL292
Finding matches for 138 3194 phs000007.v26.pht000021.v3.FL388
Finding matches for 139 3195 phs000007.v26.pht000021.v3.FL389
Finding matches for 140 3196 phs000007.v26.pht000021.v3.FL392
Finding matches for 141 3334 phs000007.v26.pht000021.v3.FL186
Finding matches for 142 3344 phs000007.v26.pht000021.v3.FL196
Finding matches for 143 3345 phs000007.v26.pht000021.v3.FL197
Finding matches for 144 3453 phs000007.v26.pht000021.v3.FL332
Finding matches for 145 3506 phs000007.v26.pht000021.v3.FL393
Finding matches for 146 3518 phs000007.v26.pht000021.v3.FL406
Finding matches for 147 3519 phs000007.v26.pht000021.v3.FL407
Finding matches for 148 3581 phs000007.v26.pht000021.v3.FL517
Findin

Finding matches for 266 7453 phs000007.v26.pht000031.v7.B355
Finding matches for 267 7455 phs000007.v26.pht000031.v7.B357
Finding matches for 268 7857 phs000007.v26.pht000032.v6.shareid
Finding matches for 269 7920 phs000007.v26.pht000032.v6.C67
Finding matches for 270 7930 phs000007.v26.pht000032.v6.C77
Finding matches for 271 7931 phs000007.v26.pht000032.v6.C78
Finding matches for 272 8017 phs000007.v26.pht000032.v6.C184
Finding matches for 273 8018 phs000007.v26.pht000032.v6.C185
Finding matches for 274 8058 phs000007.v26.pht000032.v6.C225
Finding matches for 275 8108 phs000007.v26.pht000032.v6.C286
Finding matches for 276 8109 phs000007.v26.pht000032.v6.C287
Finding matches for 277 8111 phs000007.v26.pht000032.v6.C289
Finding matches for 278 8112 phs000007.v26.pht000032.v6.C290
Finding matches for 279 8124 phs000007.v26.pht000032.v6.C302
Finding matches for 280 8125 phs000007.v26.pht000032.v6.C303
Finding matches for 281 8152 phs000007.v26.pht000032.v6.C330
Finding matches for 282 

...

Finding matches for 1289 73628 phs000280.v3.pht004122.v1.LIPB01A
Finding matches for 1290 73632 phs000280.v3.pht004122.v1.LIPB03A
Finding matches for 1291 73636 phs000280.v3.pht004122.v1.GWAS_ID
Finding matches for 1292 73641 phs000280.v3.pht004123.v1.LIPC1A
Finding matches for 1293 73643 phs000280.v3.pht004123.v1.LIPC3A
Finding matches for 1294 73645 phs000280.v3.pht004123.v1.LIPC5
Finding matches for 1295 73648 phs000280.v3.pht004123.v1.GWAS_ID
Finding matches for 1296 73656 phs000280.v3.pht004124.v1.LIPD1A
Finding matches for 1297 73658 phs000280.v3.pht004124.v1.LIPD3A
Finding matches for 1298 73666 phs000280.v3.pht004124.v1.GWAS_ID
Finding matches for 1299 73723 phs000280.v3.pht004125.v1.GWAS_ID
Finding matches for 1300 73780 phs000280.v3.pht004126.v1.GWAS_ID
Finding matches for 1301 73792 phs000280.v3.pht004127.v1.GWAS_ID
Finding matches for 1302 73817 phs000280.v3.pht004128.v1.GWAS_ID
Finding matches for 1303 73904 phs000280.v3.pht004129.v1.GWAS_ID
Finding matches for 1304 73993 

Finding matches for 1417 76411 phs000280.v3.pht004198.v1.GWAS_ID
Finding matches for 1418 76430 phs000280.v3.pht004199.v1.GWAS_ID
Finding matches for 1419 76449 phs000280.v3.pht004200.v1.GWAS_ID
Finding matches for 1420 76574 phs000280.v3.pht004201.v1.GWAS_ID
Finding matches for 1421 76698 phs000280.v3.pht004202.v1.GWAS_ID
Finding matches for 1422 76820 phs000280.v3.pht004203.v1.GWAS_ID
Finding matches for 1423 76876 phs000280.v3.pht004204.v1.GWAS_ID
Finding matches for 1424 76945 phs000280.v3.pht004205.v1.GWAS_ID
Finding matches for 1425 76946 phs000280.v3.pht004206.v1.TCAL
Finding matches for 1426 76976 phs000280.v3.pht004206.v1.ALCO
Finding matches for 1427 76978 phs000280.v3.pht004206.v1.CAFF
Finding matches for 1428 76982 phs000280.v3.pht004206.v1.SODI
Finding matches for 1429 77023 phs000280.v3.pht004206.v1.GWAS_ID
Finding matches for 1430 77131 phs000280.v3.pht004207.v1.GWAS_ID
Finding matches for 1431 78460 phs000280.v3.pht004208.v1.GWAS_ID
Finding matches for 1432 78479 phs000

Finding matches for 1545 87924 phs000209.v13.pht001123.v7.sidno
Finding matches for 1546 87929 phs000209.v13.pht001123.v7.mi
Finding matches for 1547 87931 phs000209.v13.pht001123.v7.mitt
Finding matches for 1548 87946 phs000209.v13.pht001123.v7.chf
Finding matches for 1549 87948 phs000209.v13.pht001123.v7.chftt
Finding matches for 1550 87952 phs000209.v13.pht001123.v7.strk
Finding matches for 1551 87957 phs000209.v13.pht001123.v7.strktt
Finding matches for 1552 87961 phs000209.v13.pht001123.v7.dth
Finding matches for 1553 87964 phs000209.v13.pht001123.v7.dthtt
Finding matches for 1554 87965 phs000209.v13.pht001123.v7.chdh
Finding matches for 1555 87966 phs000209.v13.pht001123.v7.chdhtt
Finding matches for 1556 87967 phs000209.v13.pht001123.v7.chda
Finding matches for 1557 87968 phs000209.v13.pht001123.v7.chdatt
Finding matches for 1558 87969 phs000209.v13.pht001123.v7.cvdh
Finding matches for 1559 87970 phs000209.v13.pht001123.v7.cvdhtt
Finding matches for 1560 87971 phs000209.v13.pht

Finding matches for 1672 104263 phs000209.v13.pht003373.v1.sidno
Finding matches for 1673 104335 phs000209.v13.pht003374.v1.sidno
Finding matches for 1674 104339 phs000209.v13.pht003375.v1.sidno
Finding matches for 1675 104341 phs000209.v13.pht003376.v1.sidno
Finding matches for 1676 104539 phs000209.v13.pht003377.v1.sidno
Finding matches for 1677 104736 phs000209.v13.pht003378.v1.sidno
Finding matches for 1678 104743 phs000209.v13.pht003379.v1.sidno
Finding matches for 1679 104777 phs000209.v13.pht003479.v1.sidno
Finding matches for 1680 104780 phs000209.v13.pht003604.v2.sidno
Finding matches for 1681 104817 phs000209.v13.pht003605.v1.sidno
Finding matches for 1682 104825 phs000209.v13.pht003606.v1.sidno
Finding matches for 1683 104833 phs000209.v13.pht003607.v1.sidno
Finding matches for 1684 104841 phs000209.v13.pht003608.v1.sidno
Finding matches for 1685 104849 phs000209.v13.pht003609.v1.sidno
Finding matches for 1686 104857 phs000209.v13.pht003610.v1.sidno
Finding matches for 1687 

KeyError: 'score_desc'

In [ ]:
obs_scores_tfmcdf = calc_scores_doc_cols(obs_data_file, {"desc": ['variable_description']}, ref_id_col, obs_man_file, mult_corpora=True, corpora_col='study')

In [25]:
#TO DO add standard and add same above but for clinical trials

#doc_col = list("var_desc_1”, “units_1", “var_coding_counts_distribution_1")
score_file = 'tests/test_var_similarity_scores_rank_data.csv'

In [60]:
result = obs_scores_tfidf[0]
result

,reference var,paired var,score_desc
0,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001452.v1.GRADE01,1.000000
1,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001450.v1.GRADE01,1.000000
2,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001490.v1.GRADE01,1.000000
3,phs000007.v26.pht000009.v2.MF5,phs000007.v26.pht000031.v7.B43,0.870514
4,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001118.v8.momschl2,0.657877
5,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001119.v8.momschl3,0.657877
6,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001118.v8.dadschl2,0.653778
7,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001119.v8.dadschl3,0.653778
8,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001116.v10.educ1,0.613257
9,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht003087.v1.educ1,0.613257


In [61]:
result = result[result["score_desc"] != 0]
result = result.rename(columns={"score_desc":"score"})

In [62]:
score_file = 'tests/test_var_similarity_scores_rank_data.csv'
score_data = pd.read_csv(score_file)
score_data = score_data[score_data["concept"] == "Age (years)"]
score_data

,concept,dbGaP_studyID_datasetID_varID_1,dbGaP_studyID_datasetID_1,dbGaP_studyID_datasetID_varID_2,dbGaP_studyID_datasetID_2,correctMatchTrue,score_desc,rank_score_desc,score_codeLab_relativeDist,rank_score_codeLab_relativeDist,Unnamed: 10
53775,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.age1c,phs000209.v13.pht001111.v4,True,0.785504,1.0,0.600015,16.0,NaN
53776,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.agequit1,phs000209.v13.pht001111.v4,False,0.324743,14.0,0.782636,2.0,NaN
53777,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.cholage1,phs000209.v13.pht001111.v4,False,0.307649,21.0,0.788611,1.0,NaN
53778,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.hystage1,phs000209.v13.pht001111.v4,False,0.456302,3.0,0.608377,14.0,NaN
53779,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.aspsage1,phs000209.v13.pht001111.v4,False,0.303891,22.0,0.739248,3.0,NaN
53780,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.pipageq1,phs000209.v13.pht001111.v4,False,0.311395,17.0,0.689516,4.0,NaN
53781,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.dbage1,phs000209.v13.pht001111.v4,False,0.308856,20.0,0.680037,7.0,NaN
53782,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.bphxage1,phs000209.v13.pht001111.v4,False,0.298671,25.0,0.688714,5.0,NaN
53783,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.aspeage1,phs000209.v13.pht001111.v4,False,0.296889,26.0,0.686862,6.0,NaN
53784,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.cgrageq1,phs000209.v13.pht001111.v4,False,0.311582,16.0,0.657114,8.0,NaN


In [64]:
score_comb = pd.merge(score_data, result, how='left', left_on=[ 'dbGaP_studyID_datasetID_varID_1', 'dbGaP_studyID_datasetID_varID_2' ], right_on=[ 'reference var', 'paired var' ]).round(6)
score_comb
dt = score_comb[["score_desc", "score", 'dbGaP_studyID_datasetID_varID_1', 'dbGaP_studyID_datasetID_varID_2', "reference var", "paired var"]]
dt = dt[~np.isnan(dt["score"])]
#dt = dt[dt['score'] != dt['score_desc']]
dt.sort_values("score")

,score_desc,score,dbGaP_studyID_datasetID_varID_1,dbGaP_studyID_datasetID_varID_2,reference var,paired var
56192,0.020119,0.020119,phs000287.v5.pht001466.v1.AGE2,phs000209.v13.pht001116.v10.total_cis_e1c,phs000287.v5.pht001466.v1.AGE2,phs000209.v13.pht001116.v10.total_cis_e1c
53533,0.024988,0.024988,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht001111.v4.cepgfr1c
53949,0.025264,0.025264,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003091.v3.cepgfr5c
53834,0.025264,0.025264,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003086.v3.cepgfr5c
48084,0.025491,0.025491,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht001111.v4.cepgfr1c
51986,0.025580,0.025580,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht001111.v4.cepgfr1c
48452,0.025773,0.025773,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003086.v3.cepgfr5c
48580,0.025773,0.025773,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003091.v3.cepgfr5c
52431,0.025863,0.025863,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003091.v3.cepgfr5c
52310,0.025863,0.025863,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003086.v3.cepgfr5c
